### Importing libraries

In [371]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import argparse

### Importing function for calculating geo distance

In [372]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

def distance_meters_from_mercator(start, finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return start.distance(finish)

### Extracting raw data

#### Bicimad

Importing bicimad dataset from a csv:

In [373]:
bicimad_stations=pd.read_csv('../data/bicimad_stations.csv')

Taking only the intersting columns needed for the project:

In [374]:
bicimad_stations=bicimad_stations[["address","geometry_coordinates","name","dock_bikes"]]

Calculating coordinates:

In [375]:
bicimad_stations[["long_finish","lat_finish"]]=bicimad_stations['geometry_coordinates'].str.split(',', 1, expand=True)

In [376]:
bicimad_stations["lat_finish"]=bicimad_stations["lat_finish"].str.replace("]","", regex=True)

In [377]:
bicimad_stations["long_finish"]=bicimad_stations["long_finish"].str.replace("[","", regex=True)

Renaming field names:

In [378]:
bicimad_stations = bicimad_stations.rename(columns={'address': 'Station location', 'name': 'BiciMAD station'})

Deleting not needed columns:

In [379]:
bicimad_stations = bicimad_stations.drop(columns='geometry_coordinates')

In [380]:
bicimad_stations

,Station location,BiciMAD station,dock_bikes,long_finish,lat_finish
0,Puerta del Sol nº 1,Puerta del Sol A,15,-3.7018341,40.4172137
1,Puerta del Sol nº 1,Puerta del Sol B,22,-3.701602938060457,40.41731271011562
2,Calle Miguel Moya nº 1,Miguel Moya,12,-3.7058415,40.4205886
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,4,-3.7069171,40.4302937
4,Calle Manuela Malasaña nº 5,Malasaña,12,-3.7025875,40.4285524
...,...,...,...,...,...
259,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,0,-3.72699,40.44375
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,12,-3.72693,40.44342
261,Calle José Antonio Novais frente al nº 12,Facultad Biología,9,-3.7272945,40.4483322
262,Avenida Complutense nº 23,Facultad Derecho,5,-3.72937,40.45109


In [381]:
bicimad_stations[["long_finish", "lat_finish"]] = bicimad_stations[["long_finish", 
                                                                   "lat_finish"]].apply(pd.to_numeric)

In [382]:
%%time
coordinates_bicimad=bicimad_stations.apply(lambda x: to_mercator(x['lat_finish'],
                                                              x['long_finish']), axis=1)

CPU times: user 7.94 s, sys: 675 ms, total: 8.61 s
Wall time: 8.78 s


In [383]:
bicimad_stations["coordinates_bicimad"]=coordinates_bicimad

In [384]:
bicimad_stations.drop(['long_finish','lat_finish'], axis=1, inplace=True)

In [385]:
bicimad_stations

,Station location,BiciMAD station,dock_bikes,coordinates_bicimad
0,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol nº 1,Puerta del Sol B,22,POINT (4499234.670122574 -412347.498528782)
2,Calle Miguel Moya nº 1,Miguel Moya,12,POINT (4499599.340516399 -412820.3206106317)
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,4,POINT (4500679.707306497 -412940.3068148262)
4,Calle Manuela Malasaña nº 5,Malasaña,12,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...
259,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,0,POINT (4502177.655770458 -415179.5220676512)
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,12,POINT (4502140.920338497 -415172.8287427756)
261,Calle José Antonio Novais frente al nº 12,Facultad Biología,9,POINT (4502687.743941171 -415213.4906984274)
262,Avenida Complutense nº 23,Facultad Derecho,5,POINT (4502994.74083288 -415445.024322674)


#### Interesting Points

Defining public url for accesing to the datasets:

In [386]:
url_interest_points = 'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

In [387]:
response_interest_points = requests.get(url_interest_points)

In [388]:
response_interest_points=response.json()['@graph']

Converting json to pandas dataframe:

In [389]:
interest_points = pd.json_normalize(response_interest_points)

In [390]:
interest_points=interest_points[["title","address.street-address","location.latitude","location.longitude"]]

In [391]:
interest_points = interest_points.rename(columns=
{'title': 'Place of interest', 'address.street-address': 'Place address',
 'location.latitude': 'lat_start','location.longitude': 'long_start'})

In [392]:
interest_points["Type of place"] = "Monumentos de la ciudad de Madrid" 

In [393]:
interest_points

,Place of interest,Place address,lat_start,long_start,Type of place
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,Monumentos de la ciudad de Madrid
1,A los Abuelos,C Alicún,40.479812,-3.661719,Monumentos de la ciudad de Madrid
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,Monumentos de la ciudad de Madrid
3,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,Monumentos de la ciudad de Madrid
4,Abstracta I,,40.421746,-3.662293,Monumentos de la ciudad de Madrid
...,...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,Monumentos de la ciudad de Madrid
1845,Yawar Fiesta,,NaN,NaN,Monumentos de la ciudad de Madrid
1846,'Sin título',,NaN,NaN,Monumentos de la ciudad de Madrid


In [394]:
interest_points[["long_start", "lat_start"]] = interest_points[["long_start", 
                                                                   "lat_start"]].apply(pd.to_numeric)

In [395]:
%%time
coordinates_interest_points=interest_points.apply(lambda x: to_mercator(x['lat_start'],
                                                              x['long_start']), axis=1)

CPU times: user 49.1 s, sys: 3.3 s, total: 52.4 s
Wall time: 52.8 s


In [396]:
interest_points["coordinates_interest_points"]=coordinates_interest_points

In [397]:
interest_points.drop(['long_start','lat_start'], axis=1, inplace=True)

In [398]:
interest_points

,Place of interest,Place address,Type of place,coordinates_interest_points
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501326.057070401 -415739.3795281866)
1845,Yawar Fiesta,,Monumentos de la ciudad de Madrid,POINT (nan nan)
1846,'Sin título',,Monumentos de la ciudad de Madrid,POINT (nan nan)


Using a csv directly downloaded in the repo:

### Joining Data

Joining both tables with all posibilities for calculating the distance:

In [399]:
interest_points_bicimad = bicimad_stations.assign(key=0).merge(interest_points.assign(key=0), how='left', on = 'key')

In [400]:
interest_points_bicimad.drop('key', axis=1, inplace=True)

In [401]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,coordinates_bicimad,Place of interest,Place address,Type of place,coordinates_interest_points
0,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...,...,...,...,...
487867,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)
487868,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501326.057070401 -415739.3795281866)
487869,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),Yawar Fiesta,,Monumentos de la ciudad de Madrid,POINT (nan nan)
487870,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),'Sin título',,Monumentos de la ciudad de Madrid,POINT (nan nan)


### Calculating distance

In [402]:
%%time
distance=interest_points_bicimad.apply(lambda x: distance_meters_from_mercator(x['coordinates_interest_points'],
                                                              x['coordinates_bicimad']), axis=1)

CPU times: user 7.61 s, sys: 103 ms, total: 7.71 s
Wall time: 7.83 s


In [403]:
interest_points_bicimad["distance"]=distance

In [404]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,coordinates_bicimad,Place of interest,Place address,Type of place,coordinates_interest_points,distance
0,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891),1.215208e+04
1,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754),8.281468e+03
2,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676),6.773300e+03
3,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985),5.201424e+02
4,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323),4.439554e+03
...,...,...,...,...,...,...,...,...,...
487867,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701),4.378590e+03
487868,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501326.057070401 -415739.3795281866),4.380043e+03
487869,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),Yawar Fiesta,,Monumentos de la ciudad de Madrid,POINT (nan nan),1.797693e+308
487870,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),'Sin título',,Monumentos de la ciudad de Madrid,POINT (nan nan),1.797693e+308


In [405]:
interest_points_bicimad.drop(['coordinates_bicimad','coordinates_interest_points'], axis=1, inplace=True)

### Filtering by the minimun distant:

In [406]:
interest_points_bicimad_min=interest_points_bicimad.groupby(['Place of interest'])['distance'].min()

In [407]:
interest_points_bicimad_min = pd.DataFrame(interest_points_bicimad_min)
interest_points_bicimad_min

,distance
Place of interest,
'Sin título',1.797693e+308
A Neruda-El Ser alado,2.012573e+03
A las víctimas del Holocausto,5.559887e+03
A los Abuelos,2.633108e+03
AICO,1.101270e+02
...,...
Álvaro de Bazán,1.023284e+02
Ángeles de la Paz,3.265693e+02
Árbol de la Vida,5.559887e+03


In [408]:
nearest_BiciMAD_station=interest_points_bicimad.merge(interest_points_bicimad_min, how='inner', on = ['Place of interest','distance'])
nearest_BiciMAD_station

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance
0,Puerta del Sol nº 1,Puerta del Sol A,15,Escuela T.S. Ingenieros de Montes I,CALLE Ramiro de Maeztu,Monumentos de la ciudad de Madrid,1.797693e+308
1,Puerta del Sol nº 1,Puerta del Sol B,22,Escuela T.S. Ingenieros de Montes I,CALLE Ramiro de Maeztu,Monumentos de la ciudad de Madrid,1.797693e+308
2,Calle Miguel Moya nº 1,Miguel Moya,12,Escuela T.S. Ingenieros de Montes I,CALLE Ramiro de Maeztu,Monumentos de la ciudad de Madrid,1.797693e+308
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,4,Escuela T.S. Ingenieros de Montes I,CALLE Ramiro de Maeztu,Monumentos de la ciudad de Madrid,1.797693e+308
4,Calle Manuela Malasaña nº 5,Malasaña,12,Escuela T.S. Ingenieros de Montes I,CALLE Ramiro de Maeztu,Monumentos de la ciudad de Madrid,1.797693e+308
...,...,...,...,...,...,...,...
15669,Avenida Complutense nº 23,Facultad Derecho,5,Rejas del arroyo Prado del Rey,CARRETERA HUMERA 15,Monumentos de la ciudad de Madrid,5.529008e+03
15670,Avenida Complutense nº 23,Facultad Derecho,5,Roberto Domingo,PARQUE CASA DE CAMPO,Monumentos de la ciudad de Madrid,3.660573e+03
15671,Avenida Complutense nº 23,Facultad Derecho,5,San Fernando (Fernando III el Santo),,Monumentos de la ciudad de Madrid,2.181283e+03
15672,Avenida Complutense nº 23,Facultad Derecho,5,Santa Bárbara,Pte Puente de San Fernando,Monumentos de la ciudad de Madrid,2.175340e+03


In [409]:
nearest_BiciMAD_station[(nearest_BiciMAD_station["Place of interest"]=="A las víctimas del Holocausto") | (nearest_BiciMAD_station["Place of interest"]=="A los Abuelos")]

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance
15392,Calle Carlota O'Neill nº 32,Carlota O’Neill,1,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
15543,Paseo de la Castellana nº 298,Manuel Caldeiro,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767


### Saving a csv with the results:

In [364]:
nearest_BiciMAD_station.to_csv('../data/nearest_BiciMAD_station.csv')

### User options

In [368]:
def sum_function(a,b):
    return a+b

In [369]:
def product_function(a,b):
    return a*b

In [ ]:
# Argument parser function

def argument_parser():
    parser = argparse.ArgumentParser(description='Set operation type')
    parser.add_argument("-f", "--function", help="function type. Options are: 'sum' and 'multiply'" , type=str)
    args = parser.parse_args()
    return args